In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Mass"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
503: 3988
75: 3988
865: 3988
822: 3988
3988: 3988
605: 3988
42: 3988
291: 3988
435: 3988
90: 3988
141: 3988
83: 3988
306: 3988
16: 3988
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
3976/3988 [============================>.] - ETA: 0s - loss: 4.8180 - leftLayer1_loss: 0.0857 - midLayer1_loss: 1.3948 - rightLayer1_loss: 0.9063 - leftLayer2_loss: 0.0567 - midLayer2_loss: 1.5425 - rightLayer2_loss: 0.8319
Epoch 00006: val_loss improved from 4.78935 to 4.73237, saving model to ./experiments/0.009_weights.h5
3988/3988 [==============================] - 10s 3ms/step - loss: 4.8179 - leftLayer1_loss: 0.0857 - midLayer1_loss: 1.3948 - rightLayer1_loss: 0.9062 - leftLayer2_loss: 0.0567 - midLayer2_loss: 1.5426 - rightLayer2_loss: 0.8319 - val_loss: 4.7324 - val_leftLayer1_loss: 0.0826 - val_midLayer1_loss: 1.3795 - val_rightLayer1_loss: 0.8766 - val_leftLayer2_loss: 0.0713 - val_midLayer2_loss: 1.4023 - val_rightLayer2_loss: 0.9201
Epoch 7/11
3967/3988 [============================>.] - ETA: 0s - loss: 4.7833 - leftLayer1_loss: 0.0808 - midLayer1_loss: 1.3951 - rightLayer1_loss: 0.8850 - leftLayer2_loss: 0.0526 - midLayer2_loss: 1.5443 - rightLayer2_loss: 0.8255

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 3988 steps, validate for 625 steps
Epoch 1/11
3987/3988 [============================>.] - ETA: 0s - loss: 6.0210 - leftLayer1_loss: 0.1158 - midLayer1_loss: 1.3835 - rightLayer1_loss: 1.6077 - leftLayer2_loss: 0.1146 - midLayer2_loss: 1.4336 - rightLayer2_loss: 1.3658
Epoch 00001: val_loss improved from inf to 5.58991, saving model to ./experiments/0.009999999999999998_weights.h5
3988/3988 [==============================] - 11s 3ms/step - loss: 6.0207 - leftLayer1_loss: 0.1158 - midLayer1_loss: 1.3835 - rightLayer1_loss: 1.6076 - leftLayer2_loss: 0.1146 - midLayer2_loss: 1.4336 - rightLayer2_loss: 1.3657 - val_loss: 5.5899 - val_leftLayer1_loss: 0.1111 - val_midLayer1_loss: 1.3695 - val_rightLayer1_loss: 1.3845 - val_leftLayer2_loss: 0.1103 - val_midLayer2_loss: 1.3427 - val_rightLayer2_loss: 1.

3982/3988 [============================>.] - ETA: 0s - loss: 4.5714 - leftLayer1_loss: 0.0627 - midLayer1_loss: 1.3830 - rightLayer1_loss: 0.8397 - leftLayer2_loss: 0.0431 - midLayer2_loss: 1.4318 - rightLayer2_loss: 0.8110
Epoch 00011: val_loss improved from 4.53993 to 4.52176, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Mass: 0.6600254217271858

auprc 0Mass: 0.08628308259280595

auroc 1Mass: 0.2864176704830335

auprc 1Mass: 0.03226412456465011

auroc 2Mass: 0.654224005403412

auprc 2Mass: 0.09631971045701654

auroc 3Mass: 0.45817697002846736

auprc 3Mass: 0.04474688578495345

auroc 4Mass: 0.48694187188443994

auprc 4Mass: 0.04924851978818534

auroc 5Mass: 0.4163903012070659

auprc 5Mass: 0.03970315552597039

mean auroc: 0.4936960401222674

mean auprc: 0.05809424645226363

max auroc: 0.6600254217271858

max auprc: 0.09631971045701654

142.

Epoch 4/11
3979/3988 [============================>.] - ETA: 0s - loss: 4.8568 - leftLayer1_loss: 0.1004 - midLayer1_loss: 1.4384 - rightLayer1_loss: 1.0097 - leftLayer2_loss: 0.0695 - midLayer2_loss: 1.3797 - rightLayer2_loss: 0.8590
Epoch 00004: val_loss improved from 5.00208 to 4.87248, saving model to ./experiments/0.010999999999999998_weights.h5
3988/3988 [==============================] - 10s 3ms/step - loss: 4.8563 - leftLayer1_loss: 0.1004 - midLayer1_loss: 1.4383 - rightLayer1_loss: 1.0095 - leftLayer2_loss: 0.0695 - midLayer2_loss: 1.3797 - rightLayer2_loss: 0.8588 - val_loss: 4.8725 - val_leftLayer1_loss: 0.0972 - val_midLayer1_loss: 1.4300 - val_rightLayer1_loss: 0.9630 - val_leftLayer2_loss: 0.0801 - val_midLayer2_loss: 1.3201 - val_rightLayer2_loss: 0.9821
Epoch 5/11
3971/3988 [============================>.] - ETA: 0s - loss: 4.7681 - leftLayer1_loss: 0.0943 - midLayer1_loss: 1.4387 - rightLayer1_loss: 0.9552 - leftLayer2_loss: 0.0619 - midLayer2_loss: 1.3754 - rightLaye

...

3986/3988 [============================>.] - ETA: 0s - loss: 4.4714 - leftLayer1_loss: 0.0654 - midLayer1_loss: 1.3469 - rightLayer1_loss: 0.8383 - leftLayer2_loss: 0.0438 - midLayer2_loss: 1.3663 - rightLayer2_loss: 0.8106
Epoch 00011: val_loss improved from 4.49056 to 4.47206, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Mass: 0.40516365625349626

auprc 0Mass: 0.03927204058192476

auroc 1Mass: 0.3977325352527048

auprc 1Mass: 0.04116284354985536

auroc 2Mass: 0.506572894264676

auprc 2Mass: 0.0497551898141976

auroc 3Mass: 0.49927721077864656

auprc 3Mass: 0.051365763619218824

auroc 4Mass: 0.4215225480568021

auprc 4Mass: 0.04177850513173886

auroc 5Mass: 0.4336061559116393

auprc 5Mass: 0.04226904943632876

mean auroc: 0.4439791667529942

mean auprc: 0.044267232022210695

max auroc: 0.506572894264676

max auprc: 0.051365763619218824

142

Epoch 4/11
3975/3988 [============================>.] - ETA: 0s - loss: 4.7853 - leftLayer1_loss: 0.0993 - midLayer1_loss: 1.3847 - rightLayer1_loss: 0.9801 - leftLayer2_loss: 0.0736 - midLayer2_loss: 1.3873 - rightLayer2_loss: 0.8603
Epoch 00004: val_loss improved from 4.94322 to 4.81732, saving model to ./experiments/0.012999999999999996_weights.h5
3988/3988 [==============================] - 11s 3ms/step - loss: 4.7851 - leftLayer1_loss: 0.0993 - midLayer1_loss: 1.3847 - rightLayer1_loss: 0.9800 - leftLayer2_loss: 0.0736 - midLayer2_loss: 1.3872 - rightLayer2_loss: 0.8603 - val_loss: 4.8173 - val_leftLayer1_loss: 0.0956 - val_midLayer1_loss: 1.3760 - val_rightLayer1_loss: 0.9384 - val_leftLayer2_loss: 0.0851 - val_midLayer2_loss: 1.3302 - val_rightLayer2_loss: 0.9921
Epoch 5/11
3976/3988 [============================>.] - ETA: 0s - loss: 4.7029 - leftLayer1_loss: 0.0926 - midLayer1_loss: 1.3845 - rightLayer1_loss: 0.9309 - leftLayer2_loss: 0.0654 - midLayer2_loss: 1.3851 - rightLaye

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 3988 steps, validate for 625 steps
Epoch 1/11
3982/3988 [============================>.] - ETA: 0s - loss: 5.9679 - leftLayer1_loss: 0.1149 - midLayer1_loss: 1.3251 - rightLayer1_loss: 1.6129 - leftLayer2_loss: 0.1224 - midLayer2_loss: 1.4316 - rightLayer2_loss: 1.3609
Epoch 00001: val_loss improved from inf to 5.54907, saving model to ./experiments/0.013999999999999995_weights.h5
3988/3988 [==============================] - 10s 3ms/step - loss: 5.9669 - leftLayer1_loss: 0.1149 - midLayer1_loss: 1.3251 - rightLayer1_loss: 1.6125 - leftLayer2_loss: 0.1224 - midLayer2_loss: 1.4316 - rightLayer2_loss: 1.3604 - val_loss: 5.5491 - val_leftLayer1_loss: 0.1102 - val_midLayer1_loss: 1.3171 - val_rightLayer1_loss: 1.3831 - val_leftLayer2_loss: 0.1145 - val_midLayer2_loss: 1.3561 - val_rightLayer2_loss: 1.

3975/3988 [============================>.] - ETA: 0s - loss: 4.5104 - leftLayer1_loss: 0.0629 - midLayer1_loss: 1.3257 - rightLayer1_loss: 0.8392 - leftLayer2_loss: 0.0444 - midLayer2_loss: 1.4266 - rightLayer2_loss: 0.8117
Epoch 00011: val_loss improved from 4.49644 to 4.47836, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Mass: 0.5602425526977693

auprc 0Mass: 0.061913283631836624

auroc 1Mass: 0.3366392766720949

auprc 1Mass: 0.03428581279795849

auroc 2Mass: 0.5251588288187495

auprc 2Mass: 0.05346923996304259

auroc 3Mass: 0.6018118833625465

auprc 3Mass: 0.06359309577401091

auroc 4Mass: 0.29376100675840866

auprc 4Mass: 0.03255300158767239

auroc 5Mass: 0.44407358833791216

auprc 5Mass: 0.041959142267033823

mean auroc: 0.46028118944124685

mean auprc: 0.04796226267025914

max auroc: 0.6018118833625465

max auprc: 0.06359309577401091



Epoch 4/11
3980/3988 [============================>.] - ETA: 0s - loss: 4.8040 - leftLayer1_loss: 0.0945 - midLayer1_loss: 1.4202 - rightLayer1_loss: 0.9570 - leftLayer2_loss: 0.0681 - midLayer2_loss: 1.4088 - rightLayer2_loss: 0.8553
Epoch 00004: val_loss improved from 4.91255 to 4.79372, saving model to ./experiments/0.014999999999999994_weights.h5
3988/3988 [==============================] - 10s 2ms/step - loss: 4.8033 - leftLayer1_loss: 0.0945 - midLayer1_loss: 1.4202 - rightLayer1_loss: 0.9568 - leftLayer2_loss: 0.0681 - midLayer2_loss: 1.4086 - rightLayer2_loss: 0.8552 - val_loss: 4.7937 - val_leftLayer1_loss: 0.0908 - val_midLayer1_loss: 1.4049 - val_rightLayer1_loss: 0.9194 - val_leftLayer2_loss: 0.0817 - val_midLayer2_loss: 1.3126 - val_rightLayer2_loss: 0.9843
Epoch 5/11
3976/3988 [============================>.] - ETA: 0s - loss: 4.7286 - leftLayer1_loss: 0.0876 - midLayer1_loss: 1.4204 - rightLayer1_loss: 0.9130 - leftLayer2_loss: 0.0609 - midLayer2_loss: 1.4070 - rightLaye

In [17]:
print(np.max(maxi))

0.7565029689759623
